## Observations

1. Looking at "Capomulin Regimen: s185 Average Tumor Volume VS. Timepoint", over time this drug regimen has decreased the tumor volume in mouse id s185. This suggests the drug Capomulin is successfully treating the tumor.

2. Looking at "Final Tumor Volumes Across Four Drug Regimens", Ramicane and Capomulin have smaller tumor volumes at the final timepoint, which suggests these drug regimens were more effective than the others.

3. Looking at the "Capomulin Regimen Average Tumor Volume VS. Mouse Weight" linear regression, as weight increases tumor volume is likely to increase. The correlation coefficient of .84 supports this strong relationship.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
mouse_results_df = pd.merge(mouse_metadata, study_results, how="left", on=["Mouse ID", "Mouse ID"])

## Data Preview

In [ ]:
# Display the data table for preview
mouse_results_df.head()

## Data Cleansing

In [ ]:
# Checking the number of mice.
mouse_results_df["Mouse ID"].nunique()

In [ ]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
mouse_group_df = mouse_results_df.groupby(["Mouse ID", "Timepoint"]).count()
mouse_group_df = mouse_group_df.loc[mouse_group_df["Drug Regimen"]>1, :]
mouse_group_df = mouse_group_df.reset_index()

mouseid_todrop = mouse_group_df["Mouse ID"].unique()[0]

mouseid_todrop 

In [ ]:
# Get all the data for the duplicate mouse ID. 
mouse_results_df.loc[mouse_results_df["Mouse ID"]==mouseid_todrop, :]



In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
indexes_to_drop = mouse_results_df[mouse_results_df["Mouse ID"]==mouseid_todrop].index

mouse_results_df.drop(indexes_to_drop, inplace=True)
mouse_results_df

In [ ]:
# Checking the number of mice in the clean DataFrame.

mouse_results_df["Mouse ID"].nunique()

## Summary Statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.

mouse_group = mouse_results_df.groupby(["Drug Regimen"])


#Find the mean tumor volume for each regimen
mouse_mean_df = pd.DataFrame(mouse_group["Tumor Volume (mm3)"].mean())
mouse_mean_df = mouse_mean_df.rename(columns={"Tumor Volume (mm3)": "Tumor Volume Mean"})
mouse_mean_df = mouse_mean_df.reset_index()

#Find the median tumor volume for each regimen
mouse_median_df = pd.DataFrame(mouse_group["Tumor Volume (mm3)"].median())
mouse_median_df = mouse_median_df.rename(columns={"Tumor Volume (mm3)": "Tumor Volume Median"})
mouse_median_df = mouse_median_df.reset_index()

#Find the variance tumor volume for each regimen
mouse_variance_df = pd.DataFrame(mouse_group["Tumor Volume (mm3)"].var())
mouse_variance_df = mouse_variance_df.rename(columns={"Tumor Volume (mm3)": "Tumor Volume Variance"})
mouse_variance_df = mouse_variance_df.reset_index()

#Find the standard deviation tumor volume for regimen
mouse_stdev_df = pd.DataFrame(mouse_group["Tumor Volume (mm3)"].std())
mouse_stdev_df = mouse_stdev_df.rename(columns={"Tumor Volume (mm3)": "Tumor Volume Standard Deviation"})
mouse_stdev_df = mouse_stdev_df.reset_index()


#Find the SEM tumor volume for each regimen
mouse_sem_df = pd.DataFrame(mouse_group["Tumor Volume (mm3)"].sem())
mouse_sem_df = mouse_sem_df.rename(columns={"Tumor Volume (mm3)": "Tumor Volume SEM"})
mouse_sem_df = mouse_sem_df.reset_index()

summary_df = mouse_mean_df.merge(mouse_median_df, how="outer", on="Drug Regimen")
summary_df = summary_df.merge(mouse_variance_df, how="outer", on="Drug Regimen")
summary_df = summary_df.merge(mouse_stdev_df, how="outer", on="Drug Regimen")
summary_df = summary_df.merge(mouse_sem_df, how="outer", on="Drug Regimen")


summary_df = summary_df.set_index("Drug Regimen")
summary_df

## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of measurements taken on each drug regimen using pandas.
drug_counts_df = pd.DataFrame(mouse_results_df["Drug Regimen"].value_counts())

drug_counts_df.plot(kind="bar", color="r", alpha=0.5, align="center", legend=False, width=.5)

plt.title("Measurements per Drug Regimen")
plt.ylabel("Amount of Measurements Taken")
plt.xlabel("Drug Regimen")
plt.tight_layout()
plt.show()

In [ ]:
# Generate a bar plot showing the total number of measurements taken on each drug regimen using pyplot.
x_axis = drug_counts_df.index

plt.bar(x_axis, drug_counts_df["Drug Regimen"], color="r", alpha=0.5, align = "center", width=.5)

plt.title("Measurements per Drug Regimen")
plt.xticks(rotation="vertical")
plt.ylabel("Amount of Measurements Taken")
plt.xlabel("Drug Regimen")
plt.tight_layout()
plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas
gender_df = pd.DataFrame(mouse_results_df["Sex"].value_counts())
colors = ["lightskyblue", "lightcoral"]
explode = (0.1, 0)

gender_df.plot(kind="pie", y="Sex", explode=explode, colors=colors, autopct="%1.1f%%", shadow=True, startangle=90, legend=False)
plt.axis("off")
plt.title(f"Distribution of Female and Male Mice")
plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
labels = gender_df.index
counts = gender_df["Sex"]

plt.pie(counts, explode=explode, labels=labels, colors=colors, autopct="%1.1f%%", shadow=True, startangle=90)

plt.axis("off")
plt.title(f"Distribution of Female and Male Mice")
plt.show()


## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

regimen_results_df = mouse_results_df.loc[(mouse_results_df["Drug Regimen"]=="Capomulin") 
                                          | (mouse_results_df["Drug Regimen"]=="Ramicane")
                                          | (mouse_results_df["Drug Regimen"]=="Infubinol")
                                          | (mouse_results_df["Drug Regimen"]=="Ceftamin")
                                          ,:]

# Start by getting the last (greatest) timepoint for each mouse
regimen_group = regimen_results_df.groupby(["Mouse ID"])
regimen_group_df = pd.DataFrame(regimen_group["Timepoint"].max())

# Merge this group df with the original dataframe to get the tumor volume at the last timepoint
regimen_results_df = regimen_results_df.merge(regimen_group_df, how="inner", on=["Mouse ID", "Timepoint"])

regimen_results_df


In [ ]:
# Put treatments into a list for for loop (and later for plot labels)
treatments = regimen_results_df["Drug Regimen"].unique()

# Create empty list to fill with tumor vol data (for plotting)
tumor_volume = []
tumor_volumes = []
ticks = []
tick = 0

# Calculate the IQR and quantitatively determine if there are any potential outliers. 
for treatment in treatments:
    tick += 1
    ticks.append(tick)

    # Locate the rows which contain mice on each drug and get the tumor volumes
    treatment_df = regimen_results_df.loc[regimen_results_df["Drug Regimen"]==treatment, :]
    tumor_volume = treatment_df["Tumor Volume (mm3)"]
   
    # add subset 
    tumor_volumes.append(tumor_volume)
    
    #Calculte Quartiles and IQR
    quartiles = tumor_volume.quantile([.25, .5, .75])
    lowerq = quartiles[0.25]
    upperq = quartiles[0.75]
    iqr = upperq-lowerq   
    
    # Determine outliers using upper and lower bounds
    lower_bound = lowerq - (1.5*iqr)
    upper_bound = upperq + (1.5*iqr)

    #Print Calculations for each treatment    
    print(f"{treatment} treatment results:")
    print(f"  The lower quartile of tumor volumes is: {lowerq}")
    print(f"  The upper quartile of tumor volumes is: {upperq}")
    print(f"  The interquartile range of tumor volumes is: {iqr}")
    print(f"  The median of tumor volumes is: {quartiles[0.5]}")
    print()
    print(f"  Values below {lower_bound} could be outliers.")
    print(f"  Values above {upper_bound} could be outliers.")
    print()
    print()

    

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest
fig1, ax1 = plt.subplots()
ax1.set_title(f'Final Tumor Volumes\nAcross Four Drug Regimens')
ax1.set_ylabel('Final Tumor Volume (mm3)')

flierprops = dict(marker='o', markerfacecolor='r', linestyle='none', markersize=10)
ax1.boxplot(tumor_volumes, flierprops=flierprops)
plt.xticks(ticks,treatments)
plt.show()


## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin
capomulin_results_df = mouse_results_df.loc[(mouse_results_df["Drug Regimen"]=="Capomulin"),:]

#identify a mouse treated with capomulin
capomulin_mouse = capomulin_results_df["Mouse ID"].unique()[0]

capomulin_mouse_results_df = mouse_results_df.loc[(mouse_results_df["Mouse ID"]==capomulin_mouse),:]

x_axis = capomulin_mouse_results_df["Timepoint"]
y_axis = capomulin_mouse_results_df["Tumor Volume (mm3)"]

plt.plot(x_axis, y_axis)
plt.ylabel("Tumor Volume (mm3)")
plt.xlabel("Timepoint")
plt.title(f"Capomulin Regimen: {capomulin_mouse}\nAverage Tumor Volume VS. Timepoint")
plt.show()

In [ ]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen
capomulin_group = capomulin_results_df.groupby(["Mouse ID"])

capomulin_mean_df = pd.DataFrame(capomulin_group["Tumor Volume (mm3)"].mean())
capomulin_weight_df = pd.DataFrame(capomulin_group["Weight (g)"].max())

capomulin_summary_df = capomulin_mean_df.merge(capomulin_weight_df, how="inner", on="Mouse ID")

x_axis = capomulin_summary_df["Weight (g)"]
y_axis= capomulin_summary_df["Tumor Volume (mm3)"]

plt.scatter(x_axis, y_axis)
plt.xlabel('Weight (g)')
plt.ylabel('Tumor Volume (mm3)')
plt.title("Capomulin Regimen \n Average Tumor Volume VS. Mouse Weight")
plt.show()

## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient 
# for mouse weight and average tumor volume for the Capomulin regimen

print(f"The correlation coefficient between Tumor Volume and Weight is {round(st.pearsonr(x_axis,y_axis)[0],2)}")

In [ ]:
#Linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_axis, y_axis)

#y=mx+b
regress_values = x_axis * slope + intercept
line_eq = f"y = {slope:.2f}x + {intercept:.2f}"

plt.scatter(x_axis, y_axis)
plt.plot(x_axis,regress_values,"r-")
plt.annotate(line_eq,(19,36),fontsize=15,color="red")

plt.xlabel('Weight (g)')
plt.ylabel('Tumor Volume (mm3)')
plt.title("Capomulin Regimen \n Average Tumor Volume VS. Mouse Weight")
plt.show()